In [ ]:
# import json
# import random
# from collections import Counter

# from konlpy.tag import Komoran
# komoran = Komoran()

# from yake import korea_token
# from yake.korea_token import edit_josa, edit_sentences

# from sklearn.feature_extraction.text import TfidfVectorizer 
# from collections import defaultdict

# from krwordrank.word import KRWordRank
# import math

# from sentence_transformers import SentenceTransformer

# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.metrics.pairwise import cosine_similarity

# from pandas import DataFrame
# from test_testmain import RougeScorer

In [ ]:
# 필요 라이브러리 선언
from pprint import pprint

from yake import korea_token
from yake.korea_token import edit_josa, edit_sentences
import math

from konlpy.tag import Komoran
from collections import Counter

import re
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from math import log
import numpy as np
from collections import defaultdict

import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences
import urllib.request
np.random.seed(seed=0)

from krwordrank.word import KRWordRank
from krwordrank.sentence import summarize_with_sentences
from krwordrank.word import summarize_with_keywords

from scipy.sparse import csr_matrix
from soynlp.tokenizer import MaxScoreTokenizer
from sklearn.metrics import pairwise_distances

import networkx

# from textrank import KeywordSummarizer

import itertools
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

import spacy
from spacy.lang.ko.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest

import random
from yake import yake
import pandas as pd
from pandas import DataFrame
import json
from test_testmain import RougeScorer
from yake.korea_token import edit_sentences

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
with open("valid_original.json", "r", encoding="utf8") as f:
    contents = f.read() # string 타입
    json_data = json.loads(contents)
    f.close()

news_list = []
for _ in range(1000):
    news_list.append(random.randint(0, 30000))  # max: 30122 || 30122개의 뉴스 데이터

news_set = set(news_list)
news_list = list(news_set)

# for i in range(len(news_list)):
i=0
id = None
text_list = []
topic_number = None

for j in range(0,len(json_data["documents"][i]["text"]),1):
    for f in range(0,len(json_data["documents"][i]["text"][j]),1):
        # print(json_data["documents"][i]["text"][j][f]["sentence"])
        elements = json_data["documents"][i]["text"][j][f]["sentence"] + '.'
        text_list.append(elements)

topic_number = json_data["documents"][i]["extractive"]
id = [int(json_data["documents"][0]['id'])]

text = [' '.join(text_list)]

In [ ]:
STRTexT = ' '.join(text_list)
LISTText = text_list

new_text = edit_sentences(STRTexT)
# print('원본 split_sentences :', new_text)
total_value = []
for x in range(len(new_text)):
    total_value.append(' '.join(edit_josa(new_text[x])))
TUNNEDText = total_value

In [ ]:
def noun_parser(txt:str):
    noun = komoran.nouns(txt)
    count = Counter(noun)

    # 명사 빈도 카운트q
    noun_list = count.most_common(10)
    return_list = []
    for i in noun_list:
        return_list.append(i[0])

    return return_list

In [ ]:
def tfidf_parser(txt:list):
    vectorizer = TfidfVectorizer()
    vectorizer.fit(txt)
    matrix = vectorizer.transform(txt)

    vocabulary_word_id = defaultdict(int)
        
    for idx, token in enumerate(vectorizer.get_feature_names()):
        vocabulary_word_id[token] = idx
        
    # 특징 추출 결과: {"token": value}
    result = defaultdict(str)
        
    for token in vectorizer.get_feature_names():
        result[token] = matrix[0, vocabulary_word_id[token]]
        
    # 내림차순 (중요도 high) 기준 정렬
    result = sorted(result.items(), key = lambda item: item[1], reverse = True)
    return_list = []
    for i in result:
        if i[1] == 0:
            continue
        else:
            if len(return_list) < 10:
                return_list.append(i[0])
            else:
                continue
    return return_list

In [ ]:
def wordrank_parser(txt:list):
    wordrank_extractor = KRWordRank(min_count=5, max_length=10)
    keywords, rank, graph = wordrank_extractor.extract(txt, num_keywords=100)

    def make_vocab_score(keywords, scaling=None):
        if scaling is None:
            scaling = lambda x:math.sqrt(x)
        return {word:scaling(rank) for word, rank in keywords.items()}

    keywords = make_vocab_score(keywords)
    return_list = []
    for i in keywords:
        if len(return_list) < 10:
            return_list.append(i[0])
        else:
            continue
    
    return return_list

In [ ]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

def keybert_parser(txt:list, model):
    
    tokenized_doc = komoran.pos(txt)
    tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] in ['NNG','NNP']])

    # print('품사 태깅 10개만 출력 :',tokenized_doc[:10])
    # print('명사 추출 :',tokenized_nouns)

    n_gram_range = (1, 3)
    count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
    candidates = count.get_feature_names_out()

    # print('trigram 개수: ', len(candidates))
    # print('trigram 5개 출력: ', candidates[:5])


    doc_embedding = model.encode([txt])
    candidate_embeddings = model.encode(candidates)

    top_n = 10
    distances = cosine_similarity(doc_embedding, candidate_embeddings)
    keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]

    return keywords

In [ ]:
# kw_extractor = yake.KeywordExtractor(n=[1,2,3],top=20, stoplen=2, windowsSize=1, WG=False, ReDup=False)

df_trainset = DataFrame(columns=["id", "summary"])
df_testset = DataFrame(columns=["id", "summary"])

scorer = RougeScorer()
_dataframe = DataFrame()

R1 = []
R2 = []
Rl = []

for i in range(len(news_list)):
    
    id = None
    text_list = []
    topic_number = None


    for j in range(0,len(json_data["documents"][i]["text"]),1):
        for f in range(0,len(json_data["documents"][i]["text"][j]),1):
            # print(json_data["documents"][i]["text"][j][f]["sentence"])
            elements = json_data["documents"][i]["text"][j][f]["sentence"] + '.'
            text_list.append(elements)

    topic_number = json_data["documents"][i]["extractive"]
    id = [int(json_data["documents"][0]['id'])]

    text = [' '.join(text_list)]


    STRTexT = ' '.join(text_list)
    LISTText = text_list

    new_text = edit_sentences(STRTexT)
    # print('원본 split_sentences :', new_text)
    total_value = []
    for x in range(len(new_text)):
        total_value.append(' '.join(edit_josa(new_text[x])))
    TUNNEDText = total_value


    ### 빈도수 기반 명사 추출: noun_parser(STRTexT)      	 ||	txt:str

    ### TF.IDF: tfidf_parser(TUNNEDText)    	        ||	txt:list

    ### KR-WordRank: wordrank_parser(TUNNEDText)	    ||	txt:list

    ### Korean KeyBERT" keybert_parser(STRTexT, model) 	||	txt:list

    ### return : list

    # for i in range(len(text_list)):                        # range(len(text_list)) : n, i : [0,1,...,n]
    # keywords = kw_extractor.extract_keywords(''.join(text))  # keywords : text[i]의 키워드 및 점수로 이루어진 튜플 리스트
    
    keywords = noun_parser(STRTexT)
    # keywords = tfidf_parser(TUNNEDText)
    # keywords = wordrank_parser(TUNNEDText)
    # keywords = keybert_parser(STRTexT, model)
    score = 0
    for m in range(len(text_list)):     # text_list[m] : text_list의 m번째 문장
        for l in keywords:              # keywords : text파일의 문서를 각 함수들을 돌린 후 도출되는 키워드들.
            if l in text_list[m]:
                score = score + 1*(1+0.2*len(l[0].split()))
        score_data = dict(zip(["id", "index", "score"], [i, m, score]))
        score = 0
        _dataframe = _dataframe.append(score_data, ignore_index=True)


    multi_index = _dataframe.sort_values(by=['score', 'index'], ascending=[False, True]).groupby("id").head(3)
    # print(multi_index)
    multi_index = multi_index.sort_values(by=['id','score'], ascending=[True, False])
    # print(multi_index)
    # print(multi_index[multi_index['id']==i])

    summary_number = list(multi_index[multi_index['id']==i]['index'])

    df_testset['id'] = id
    df_testset['summary'] = [[text_list[int(summary_number[0])],text_list[int(summary_number[1])], text_list[int(summary_number[2])]]]
    df_trainset['id'] = id
    df_trainset['summary'] = [[text_list[int(topic_number[0])],text_list[int(topic_number[1])], text_list[int(topic_number[2])]]]

    score_set = scorer.compute_rouge(df_trainset, df_testset)
    rouge_1 = score_set[0]
    rouge_2 = score_set[1]
    rouge_l = score_set[2]

    R1.append(rouge_1)
    R2.append(rouge_2)
    Rl.append(rouge_l)

    print(i,"번째 문단..")
    print("내가 선택한 문장은..? : ", (summary_number))
    print([text_list[int(summary_number[0])],text_list[int(summary_number[1])], text_list[int(summary_number[2])]])
    print("정답지는..! : ", topic_number)
    print([text_list[int(topic_number[0])],text_list[int(topic_number[1])], text_list[int(topic_number[2])]])
    print()

print("rouge_1의 평균점수: ", sum(R1) / len(R1))
print("rouge_2의 평균점수: ", sum(R2) / len(R2))
print("rouge_l의 평균점수: ", sum(Rl) / len(Rl))
